In [1]:
import os
os.system('ipcluster start -n 8 --profile=default & sleep 5 ; python stdout_worker.py &')

0

In [2]:

import matplotlib
%matplotlib inline

import ipyparallel as ipp
rc = ipp.Client(profile='default')
from ipyparallel import depend, require, dependent
dview = rc[:]


In [3]:
################
# GA parameters:
################
MU = 10; NGEN = 15; CXPB = 0.4555
USE_CACHED_GA = False
# about 8 models will be made, excluding rheobase search.

################
# Grid search parameters:
# only 5 models, will be made excluding rheobase search
################
npoints = 2
nparams = 10
from neuronunit.optimization.model_parameters import model_params
provided_keys = list(model_params.keys())
USE_CACHED_GS = True

In [4]:
import pickle
import numpy as np

In [ ]:
from neuronunit.optimization.nsga_object import NSGA
from neuronunit.optimization import exhaustive_search as es
from neuronunit.optimization import evaluate_as_module as eam


Getting Rheobase cached data value for from AIBS dataset 354190013
/home/jovyan/neuronunit/neuronunit/optimization/neuroelectro.pickle got here
/home/jovyan/neuronunit/neuronunit/optimization
attempting to recover from pickled file
/home/jovyan/neuronunit/neuronunit/models/__init__.py


/home/jovyan/neuronunit/neuronunit/models/backends.py:3: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/conda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/opt/conda/lib/python3.5/site-packa

In [ ]:

if USE_CACHED_GA:
    from deap import creator
    from deap import base
    from neuronunit.optimization import evaluate_as_module as eam
    NSGAO = NSGA()
    NSGAO.setnparams(nparams=nparams,provided_keys=provided_keys)
    td = eam.get_trans_dict(NSGAO.subset)
    weights = tuple([-1.0 for i in range(0,8)])
    creator.create("FitnessMin", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.FitnessMin)
    [invalid_dtc, pop, logbook, fitnesses, history, pf] = pickle.load(open('ga_dump.p','rb'))
else:
    NSGAO = NSGA()
    NSGAO.setnparams(nparams=nparams,provided_keys=provided_keys)
    invalid_dtc, pop, logbook, fitnesses, history, pf = NSGAO.main(MU, NGEN)
    with open('ga_dump.p','wb') as f:
       pickle.dump([invalid_dtc, pop, logbook, fitnesses, history, pf],f)

/home/jovyan/neuronunit/neuronunit/optimization/nsga_parallel.py:6: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/conda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/opt/conda/lib/python3.5

<ipyparallel.controller.dependency.dependent object at 0x7f797ae4b208> {'vr': array([-75.   , -71.875, -68.75 , -65.625, -62.5  , -59.375, -56.25 ,
       -53.125, -50.   ]), 'b': array([ -5.00000000e-09,  -4.68750000e-09,  -4.37500000e-09,
        -4.06250000e-09,  -3.75000000e-09,  -3.43750000e-09,
        -3.12500000e-09,  -2.81250000e-09,  -2.50000000e-09]), 'v0': array([-75.  , -71.25, -67.5 , -63.75, -60.  , -56.25, -52.5 , -48.75, -45.  ]), 'a': array([ 0.      ,  0.118125,  0.23625 ,  0.354375,  0.4725  ,  0.590625,
        0.70875 ,  0.826875,  0.945   ]), 'k': array([ 0.00063   ,  0.00072625,  0.0008225 ,  0.00091875,  0.001015  ,
        0.00111125,  0.0012075 ,  0.00130375,  0.0014    ]), 'vpeak': array([ 30.  ,  31.25,  32.5 ,  33.75,  35.  ,  36.25,  37.5 ,  38.75,  40.  ]), 'C': array([  9.00000045e-05,   9.25000046e-05,   9.50000047e-05,
         9.75000049e-05,   1.00000005e-04,   1.02500005e-04,
         1.05000005e-04,   1.07500005e-04,   1.10000005e-04]), 'vt': arra

In [ ]:
import numpy as np
import matplotlib
matplotlib.rcParams.update({'font.size':16})
import matplotlib.pyplot as plt
all_inds = history.genealogy_history.values()
sums = np.array([np.sum(ind.fitness.values) for ind in all_inds])
quads = []
for k in range(1,9):
    for i,j in enumerate(td):
        if i+k < 10:
            quads.append((td[i],td[i+k],i,i+k))
all_inds1 = list(history.genealogy_history.values())


def plot_surface(x,z):
    ab = [ (all_inds1[x],all_inds1[z]) for y in all_inds1 ]
    xs = np.array([ind[x] for ind in all_inds])
    ys = np.array([ind[z] for ind in all_inds])
    min_ys = ys[np.where(sums == np.min(sums))]
    min_xs = xs[np.where(sums == np.min(sums))]
    plt.clf()
    fig_trip, ax_trip = plt.subplots(1, figsize=(10, 5), facecolor='white')
    trip_axis = ax_trip.tripcolor(xs,ys,sums,20,norm=matplotlib.colors.LogNorm())
    plot_axis = ax_trip.plot(list(min_xs), list(min_ys), 'o', color='lightblue',label='global minima')
    fig_trip.colorbar(trip_axis, label='Sum of Objective Errors ')
    ax_trip.set_xlabel('Parameter $ b$')
    ax_trip.set_ylabel('Parameter $ a$')
    plot_axis = ax_trip.plot(list(min_xs), list(min_ys), 'o', color='lightblue')
    fig_trip.tight_layout()
    fig_trip.show()
plot_surface(4,-3)

In [ ]:

plot_surface(1,2)

In [ ]:

log = logbook
%matplotlib inline
import numpy as np
from neuronunit.optimization import get_neab
plt.clf()
plt.style.use('ggplot')
fig, axes = plt.subplots(figsize=(10, 10), facecolor='white')
gen_numbers = log.select('gen')
minimum = log.select('min')
mean = log.select('mean')
objective_labels = [ str(t) for t in get_neab.tests ]
mins_components_plot = log.select('min')
components = {}
for i in range(0,7):
    components[i] = []
    for l in mins_components_plot:
        components[i].append(l[i])
maximum = 0.0
for keys in components:

    axes.plot(
        gen_numbers,
        components[keys],
        linewidth=2,
        label=str(objective_labels[keys])
        )
    if np.max(components[keys]) > maximum:
        maximum = np.max(components[keys])

axes.set_xlim(min(gen_numbers) - 1, max(gen_numbers) + 1)
axes.set_xlabel('Generation #')
axes.set_ylabel('Sum of objectives')
axes.set_ylim([-0.1, maximum])
axes.legend()
fig.tight_layout()
fig.show()

In [ ]:
# A small administrative burden to those who intend to extend this code, in a way that 
# re-runs the grid search without using pre computed values.
# The precomputed grid run, loads in two large pickle files, as they were initially divided into halves
# Code has been re-written to use data blocks divided in thirds, and to collate the thirds into one, however,
# An appropriate pickle load for loading and collating third data blocks has not been re-written yet.
if USE_CACHED_GS:
       #dtcpopg = pickle.load(open('grid_dump.p','rb'))
    first_half = pickle.load(open('grid_dump_first_half.p','rb'))
    second_half = pickle.load(open('grid_dump_second_half.p','rb'))
    second_half.extend(first_half)
    dtcpopg = second_half
    print(dtcpopg)
else:
    from neuronunit.optimization import exhaustive_search
    grid_points = exhaustive_search.create_grid(npoints = npoints,nparams = nparams,provided_keys = provided_keys )
    dlist = list(dview.map_sync(exhaustive_search.update_dtc_pop,grid_points))
    from neuronunit.optimization import get_neab
    for d in dlist:
        d.model_path = get_neab.LEMS_MODEL_PATH
        d.LEMS_MODEL_PATH = get_neab.LEMS_MODEL_PATH
        #print(d.model_path)
    #with open('grid_dump.p','wb') as f:
     #  pickle.dump(dlist,f)
    # this is a big load on memory so divide it into thirds.

    dlist_first_third = dlist[0:int(len(dlist)/3)]
    dlist_second_third = dlist[int(len(dlist)/3):int(2*len(dlist)/3)]
    dlist_final_third = dlist[int(2*len(dlist)/3):-1]
    from neuronunit.optimization.exhaustive_search import dtc_to_rheo
    from neuronunit.optimization.exhaustive_search import parallel_method
    # uncomment if list is very small
    #if int(len(dlist)/10) == 0:
    #    dlist_first_half =[ dlist ]
    #dlist_first_3rd = compute_half(dlist_first_half)
    
    def compute_half(dlist_half):
        dlist_half = list(map(dtc_to_rheo,dlist_half))
        dlist_half = dview.map_sync(parallel_method,dlist_half)
        return dlist_half
    
    with open('grid_dump_first_3rd.p','wb') as f:
       pickle.dump(dlist_first_3rd,f)
    # Garbage collect a big memory burden.
    dlist_first_3rd = None
    dlist_second_3rd = compute_half(dlist_second_half)

    with open('grid_dump_second_3rd.p','wb') as f:
       pickle.dump(dlist_second_3rd,f)
   # Garbage collect a big memory burden.
    dlist_second_3rd = None

    dlist_final_3rd = compute_half(dlist_final_half)
    with open('grid_dump_final_3rd.p','wb') as f:
       pickle.dump(dlist_final_3rd,f)
    # Garbage collect a big memory burden.
    dlist_final_3rd = None

In [ ]:

for d in dtcpopg:
    if d.scores['RheobaseTestP'] == None:
        d.scores['RheobaseTestP'] = 10


def error(dtc):
    """
    Overall error function for a DTC
    Returns the root-mean-square error over all the tests
    """
    return np.sqrt(np.mean(np.square(list(dtc.scores.values()))))

def sorted_dtcs(dtcpop):
    """
    Returns dtc,error tuples sorted from low to high error
    """
    return sorted([(dtc,error(dtc)) for dtc in dtcpop],key=lambda x:x[1])


def sorted_history(pop):
    """
    Returns dtc,error tuples sorted from low to high error
    """
    return sorted([ind.fitness for ind in pop],key=lambda x:x[1])



minimagr_dtc, mini = sorted_dtcs(dtcpopg)[0]

minimagr_dtc_1, mini_1 = sorted_dtcs(dtcpopg)[1]
minimagr_dtc_2, miniminimagr_dtc_1,_2 = sorted_dtcs(dtcpopg)[2]
from neuronunit.optimization import create_refined_grid
grid = create_refined_grid(minimagr_dtc_1,minimagr_dtc_2)
maximagr_dtc, maxi = sorted_dtcs(dtcpopg)[-1]

In [ ]:
def pop2dtc(pop1,NSGAO):
    from deap import base, creator
    toolbox = base.Toolbox()
    NDIM = 10
    weights = tuple([1.0 for i in range(0,NDIM)])
    creator.create("FitnessMin", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.FitnessMin)
    from neuronunit.optimization import evaluate_as_module as eam
    td = eam.get_trans_dict(NSGAO.subset)
    dtc_pop = eam.update_dtc_pop(pop1,td)
    assert len(pop1) == len(dtc_pop)

    from neuronunit.optimization.exhaustive_search import dtc_to_rheo

    from neuronunit.optimization import model_parameters as modelp
    dtcpop = list(map(dtc_to_rheo,dtc_pop))
    for i,p in enumerate(pop1):
        for val in list(p.fitness.values):
            if val==-100:
                val = 100
        dtc_pop[i].scores = p.fitness.values
        dtc_pop[i].error = None
        dtc_pop[i].error = np.sqrt(np.mean(np.square(list(p.fitness.values))))
    #for i,p in enumerate(dtc_pop):
    #    print(i,p,dir(p))
    sorted_list  = sorted([(dtc,dtc.error) for dtc in dtc_pop],key=lambda x:x[1])
    return sorted_list, dtc_pop

sorted_list_pf, pareto_dtc = pop2dtc(pf,NSGAO)
sorted_list_pf, pareto_dtc = pop2dtc(pf,NSGAO)

h = list(history.genealogy_history.values())
sorted_list_h, dtc_pop = pop2dtc(h,NSGAO)
minimaga_dtc = sorted_list_pf[0][0]
maximaga_dtc = sorted_list_pf[-1][0]

# A more rigorous best candidate, uses only historical best solution, 
# especially if this is better than most evolved gene population
miniga = sorted_list_pf[0][1]
maxiga = sorted_list_pf[-1][1]
'''
A less rigorous best candidate, uses only evolved population ignores historical best solution
sdtc = sorted_dtcs(invalid_dtc)
miniga = sdtc[0][1]
maxiga = sdtc[-1][1]
'''

In [ ]:


%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
log = logbook
#plt.clf()
plt.style.use('ggplot')
fig, axes = plt.subplots(figsize=(10, 10), facecolor='white')
gen_numbers =[ i for i in range(0,len(log.select('gen'))) ]
mean = np.array([ np.sum(i) for i in log.select('avg')])
std = np.array([ np.sum(i) for i in log.select('std')])
minimum = np.array([ np.sum(i) for i in log.select('min')])
stdminus = mean - std
stdplus = mean + std
try:
    assert len(gen_numbers) == len(stdminus) == len(stdplus)
except:
    pass

axes.plot(
    gen_numbers,
    mean,
    color='black',
    linewidth=2,
    label='population average')
axes.fill_between(gen_numbers, stdminus, stdplus)
axes.plot(gen_numbers, stdminus, label='std variation lower limit')
axes.plot(gen_numbers, stdplus, label='std variation upper limit')
axes.set_xlim(np.min(gen_numbers) - 1, np.max(gen_numbers) + 1)
axes.set_xlabel('Generations')
axes.set_ylabel('Sum of objectives')
axes.set_ylim([np.min(stdminus), np.max(stdplus)])
axes.legend()
fig.tight_layout()
#fig.legend()
fig.show()


In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
log = logbook
#plt.clf()
plt.style.use('ggplot')
fig, axes = plt.subplots(figsize=(10, 10), facecolor='white')
gen_numbers =[ i for i in range(0,len(log.select('gen'))) ]
mean = np.array([ np.sum(i) for i in log.select('avg')])
std = np.array([ np.sum(i) for i in log.select('std')])
minimum = np.array([ np.sum(i) for i in log.select('min')])
best_line = np.array([ miniga for i in gen_numbers])
worst_line = np.array([ maxiga for i in gen_numbers])
print(best_line)
print(worst_line)

stdminus = mean - std
stdplus = mean + std
try:
    assert len(gen_numbers) == len(stdminus) == len(stdplus)
except:
    pass

axes.plot(
    gen_numbers,
    mean,
    color='black',
    linewidth=2,
    label='population average')
axes.fill_between(gen_numbers, stdminus, stdplus)
axes.plot(gen_numbers, best_line, linewidth=2, label='best candidate error')
axes.plot(gen_numbers, worst_line, linewidth=2, label='worst candidate error')
axes.plot(gen_numbers, stdminus, label='std variation lower limit')
axes.plot(gen_numbers, stdplus, label='std variation upper limit')
axes.set_xlim(np.min(gen_numbers) - 1, np.max(gen_numbers) + 1)
axes.set_xlabel('Generations')
axes.set_ylabel('Sum of objectives')
axes.set_ylim([-miniga-np.min(stdminus), np.max(stdplus)+maxiga])
axes.legend()
fig.tight_layout()
#fig.legend()
fig.show()

# The plot below 
# shows how population diversity (std deviation) increases, and simultaneously mean error follows a net downward
# Trajectory.

In [ ]:
def use_dtc_to_plotting(dtcpop,minimagr):
    from neuronunit.capabilities import spike_functions
    import matplotlib.pyplot as plt
    import numpy as np
    plt.clf()
    plt.style.use('ggplot')
    fig, axes = plt.subplots(figsize=(10, 10), facecolor='white')
    stored_min = []
    stored_max = []
    for dtc in dtcpop[1:-1]:
        plt.plot(dtc.tvec, dtc.vm0,linewidth=3.5, color='grey')
        stored_min.append(np.min(dtc.vm0))
        stored_max.append(np.max(dtc.vm0))
        
    from neuronunit.models.reduced import ReducedModel
    from neuronunit.optimization.get_neab import tests as T
    from neuronunit.optimization import get_neab
    from neuronunit.optimization import evaluate_as_module
    from neuronunit.optimization.evaluate_as_module import pre_format
    model = ReducedModel(get_neab.LEMS_MODEL_PATH,name=str('vanilla'),backend='NEURON')
    import neuron
    model._backend.reset_neuron(neuron)
    model.set_attrs(minimagr.attrs)
    model.rheobase = minimagr.rheobase['value']
    minimagr = pre_format(minimagr)
    parameter_list = list(minimagr.vtest.values())
    #print(parameter_list[0])
    model.inject_square_current(parameter_list[0])
    model._backend.local_run()
    assert model.get_spike_count() == 1
    print(model.get_spike_count(),bool(model.get_spike_count() == 1))
    brute_best = list(model.results['vm'])
    
    #minimagr 
    plt.plot(dtcpop[0].tvec, brute_best,linewidth=1, color='blue',label='best candidate via grid'+str(mini))
    plt.plot(dtcpop[0].tvec,dtcpop[0].vm0,linewidth=1, color='red',label='best candidate via GA'+str(miniga))
    plt.legend()
    plt.ylabel('$V_{m}$ mV')
    plt.xlabel('ms')
    plt.show()
from neuronunit import plottools
from neuronunit.plottools import dtc_to_plotting
invalid_dtc = dview.map_sync(dtc_to_plotting,dtc_pop)
use_dtc_to_plotting(invalid_dtc,minimagr_dtc)


In [ ]:
# quantize distance between minimimum error and maximum error.
quantize_distance = list(np.linspace(mini,maxi,10))

# check that the nsga error is in the bottom 1/5th of the entire error range.
print('Report: ')
print("Success" if bool(miniga < quantize_distance[1]) else "Failure")
print("The nsga error %f is in the bottom 1/5th of the entire error range" % miniga)
print("Minimum = %f; 20th percentile = %f; Maximum = %f" % (mini,quantize_distance[2],maxi))

In [ ]:

# This function reports on the deltas brute force obtained versus the GA found attributes.
from neuronunit.optimization import model_parameters as modelp
mp = modelp.model_params
for k,v in minimagr.attrs.items():
    #hvgrid = np.linspace(np.min(mp[k]),np.max(mp[k]),10)
    dimension_length = np.max(mp[k]) - np.min(mp[k])
    solution_distance_in_1D = np.abs(float(minimaga.attrs[k]))-np.abs(float(v))
    relative_distance = dimension_length/solution_distance_in_1D
    print('the difference between brute force candidates model parameters and the GA\'s model parameters:')
    print(float(minimaga.attrs[k])-float(v),minimaga.attrs[k],v,k)
    print('the relative distance scaled by the length of the parameter dimension of interest:')
    print(relative_distance)
    
    

In [ ]:
print('the difference between the bf error and the GA\'s error:')
print('grid search:')
from numpy import square, mean, sqrt
rmsg = sqrt(mean(square(list(minimagr.scores.values()))))
print(rmsg)
print('ga:')
rmsga = sqrt(mean(square(list(minimaga.scores.values()))))
print(rmsga)
